In [1]:
%pylab inline
import pandas as pd
import tensorflow as tf
import glob
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources
from tqdm import tqdm_notebook
from multiprocessing import Pool
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
np.random.seed(42)

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_table('normal_tumor_segmented_df.tsv')
FEATURE_KEYS = df.columns


In [3]:
df = pd.read_csv('normal_tumor_segmented_df_with_label.csv')
BUFFER_SIZE = len(df.index)

In [4]:
BUFFER_SIZE

293927

In [5]:

# Parameters
num_steps = 500 # Total steps to train
batch_size = 1024 # The number of samples per batch
num_classes = 2 # The 10 digits
num_features = 46 # Each image is 28x28 pixels
num_trees = 100
max_nodes = 10000


In [6]:
X = tf.placeholder(tf.float32, shape=[None, num_features])
# For random forest, labels must be integers (the class id)
Y = tf.placeholder(tf.int32, shape=[None])

# Random Forest Parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=num_trees,
                                      max_nodes=max_nodes).fill()

# Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)
# Get training graph and loss
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

# Measure the accuracy
infer_op, _, _ = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initialize the variables (i.e. assign their default value) and forest resources
init_vars = tf.group(tf.global_variables_initializer(),
                     resources.initialize_resources(resources.shared_resources()))

# Start TensorFlow session
sess = tf.Session()

# Run the initializer
sess.run(init_vars)

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'dominate_method': 'bootstrap', 'prune_every_samples': 0, 'finish_type': 0, 'early_finish_check_every_samples': 0, 'num_outputs': 1, 'max_fertile_nodes': 0, 'valid_leaf_threshold': 1, 'base_random_seed': 0, 'split_after_samples': 250, 'num_features': 46, 'num_classes': 2, 'param_file': None, 'bagging_fraction': 1.0, 'num_trees': 100, 'inference_tree_paths': False, 'split_type': 0, 'num_output_columns': 3, 'split_pruning_name': 'none', 'model_name': 'all_dense', 'split_finish_name': 'basic', 'feature_bagging_fraction': 1.0, 'regression': False, 'dominate_fraction': 0.99, 'use_running_stats_method': False, 'leaf_model_type': 0, 'pruning_type': 0, 'num_splits_to_consider': 10, 'max_nodes': 10000, 'checkpoint_stats': False, 'initialize_average_splits': False, 'collate_examples': False, 'bagged_features': None, 'stats_model_type': 0, 'split_name': 'less_or_equal', 'bagged_num_features': 46}


In [7]:
def _parse_csv(rows_string_tensor):
        """Takes the string input tensor and returns tuple of (features, labels)."""
        # Last dim is the label.        
        num_features = len(FEATURE_KEYS)
        num_columns = num_features + 1 
        columns = tf.decode_csv(rows_string_tensor,
                                record_defaults=[[0.0]] * num_features + [[0]], 
                                field_delim=',')
        return tf.cast(tf.stack(columns[:-1]), tf.float32), tf.cast(columns[-1], tf.int32)

def input_fn(file_names, batch_size):   

    """The input_fn."""
    dataset = tf.data.TextLineDataset(file_names).skip(1)
    # Skip the first line (which does not have data).
    dataset = dataset.map(_parse_csv)
    dataset = dataset.shuffle(buffer_size=BUFFER_SIZE)

    dataset = dataset.batch(batch_size)
    iterator = tf.data.Iterator.from_structure(dataset.output_types,
                                               dataset.output_shapes)
    next_batch = iterator.get_next()
    init_op = iterator.make_initializer(dataset)
    return init_op, next_batch

In [8]:
training_init_op, training_next_batch =  input_fn(['normal_tumor_segmented_df_with_label.csv'],
                                                  1024)


In [9]:
for epoch in range(num_steps):
    sess.run(training_init_op)    
    while True:
        try:
            training_features_batch, training_label_batch = sess.run(training_next_batch)
        except tf.errors.OutOfRangeError:
            break
        _, l = sess.run([train_op, loss_op], 
                        feed_dict={X: training_features_batch,
                                   Y: training_label_batch})
    acc = sess.run(accuracy_op, 
                   feed_dict={X: training_features_batch, 
                              Y: training_label_batch})
    print('Step %i, Loss: %f, Acc: %f' % (epoch, l, acc))

Step 0, Loss: -1657.439941, Acc: 0.743590
Step 1, Loss: -3348.379883, Acc: 0.820513
Step 2, Loss: -5047.959961, Acc: 0.846154
Step 3, Loss: -6737.000000, Acc: 0.871795
Step 4, Loss: -8435.200195, Acc: 0.897436
Step 5, Loss: -10001.000000, Acc: 0.820513
Step 6, Loss: -10001.000000, Acc: 0.897436
Step 7, Loss: -10001.000000, Acc: 0.846154
Step 8, Loss: -10001.000000, Acc: 0.897436
Step 9, Loss: -10001.000000, Acc: 0.846154
Step 10, Loss: -10001.000000, Acc: 0.743590
Step 11, Loss: -10001.000000, Acc: 0.820513
Step 12, Loss: -10001.000000, Acc: 0.923077
Step 13, Loss: -10001.000000, Acc: 0.871795
Step 14, Loss: -10001.000000, Acc: 0.846154
Step 15, Loss: -10001.000000, Acc: 0.846154
Step 16, Loss: -10001.000000, Acc: 0.794872
Step 17, Loss: -10001.000000, Acc: 0.794872
Step 18, Loss: -10001.000000, Acc: 0.769231
Step 19, Loss: -10001.000000, Acc: 0.846154
Step 20, Loss: -10001.000000, Acc: 0.897436
Step 21, Loss: -10001.000000, Acc: 0.717949
Step 22, Loss: -10001.000000, Acc: 0.871795
Ste

KeyboardInterrupt: 

In [ ]:
len(training_label_batch)

In [ ]:
len(training_features_batch)

In [10]:
testing_init_op, testing_next_batch =  input_fn(['test_normal_tumor_segmened_with_labels.csv'],
                                                  1024)


In [11]:
test_acc = []
sess.run(testing_init_op)    
while True:
    try:
        testing_features_batch,  testing_label_batch = sess.run(testing_next_batch)
    except tf.errors.OutOfRangeError:
        break
    acc = sess.run(accuracy_op, 
                   feed_dict={X: testing_features_batch, 
                              Y: testing_label_batch})
    test_acc.append(acc)

In [12]:
test_acc

[0.099609375,
 0.119140625,
 0.15039062,
 0.115234375,
 0.10644531,
 0.11816406,
 0.11816406,
 0.12988281,
 0.13867188,
 0.083984375,
 0.10449219,
 0.096114516]